In [1]:
# Load model directly
from transformers import AutoProcessor, AutoModelForCTC

processor = AutoProcessor.from_pretrained("arbml/wav2vec2-large-xlsr-53-arabic-egyptian")
model = AutoModelForCTC.from_pretrained("arbml/wav2vec2-large-xlsr-53-arabic-egyptian")

c:\ProgramData\anaconda3\envs\pyTorch_gpu\lib\site-packages\transformers\configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

import onnxruntime
from onnxruntime.quantization import quantize_dynamic, QuantType
import numpy as np

ModuleNotFoundError: No module named 'onnxruntime'